In [1]:
from smtutil import hiveutil, oracle_to_hive
from datetime import datetime, timedelta
import re, glob, shutil, os
from shutil import copyfile
from pathlib import Path
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
from pyspark.sql.functions import lag, lead, first, last, udf, lit, when, expr
import pyspark.sql.functions as F
import yaml
import argparse

byte_to_kb_ratio=1024

# schema
schema_raw = StructType([
    StructField("MSISDN", StringType(), True),
    StructField("IP1", StringType(), True),
    StructField("TIME_START", StringType(), True),
    StructField("IP2", StringType(), True),
    StructField("BYTE_COUNT_UP", IntegerType(), True),
    StructField("BYTE_COUNT_DOWN", IntegerType(), True),
    StructField("SSL_CERT_DOMAIN", StringType(), False),
    StructField("TIME_END", StringType(), True)])

def move_file(input_dir, output_dir, filename_format):

    for f in Path(input_dir).glob(filename_format): # grabs all files
        f.rename(Path(output_dir).joinpath(f.name)) # moves to target folder.

    print(f'[Info] Moved files from {input_dir} to {output_dir} with file format {filename_format}')
    return

In [2]:
time_stamp = '20230511'

In [3]:
# # get binary time string
# def hourly_string(l):
#     output = ''
#     for i in range(0, 24):
#         if int(i) in l:
#             output += '1'
#         else:
#             output += '0'
#     return output
# # define pyspark udf
# udf_hourly_string = F.udf(lambda x: hourly_string(x), StringType())

# get binary time string
def xmins_string(l, interval=60):
    output = ''
    for i in range(0, 24*60//interval):
        if int(i) in l:
            output += '1'
        else:
            output += '0'
    return output

# define pyspark udf
udf_hourly_string = F.udf(lambda x: xmins_string(x, 60), StringType())
udf_15mins_string = F.udf(lambda x: xmins_string(x, 15), StringType())
udf_30mins_string = F.udf(lambda x: xmins_string(x, 30), StringType())

# binary string to integer
udf_b2int = F.udf(hiveutil.binary2int)

In [4]:
# get spark session 
sc, spark = hiveutil.get_sc_spark()

In [7]:
working_dir = '/app/sna_bdmrdev01/ws-simon/data/weblog_raw_data/20230511/'
# time_stamp,
# filename_format,
# input_dir,
# pipeline_output_dir,
# derived_dir,
# byte_to_kb_ratio=1024

In [8]:
### 1. -------------------------------------- Move file to working directory --------------------------------------
# move_file(input_dir, working_dir, filename_format)

### 2. -------------------------------------- Read data file --------------------------------------
print(f'[Info] Reading Data Files.....')
print(datetime.now())

# print(f'[Info] Read csv files of timestamp {time_stamp}')
# read csv
sdf = spark.read.csv(f"{working_dir}", sep='|', header=False, schema=schema_raw)
print(f'[Info] df size: {sdf.count()}, {len(sdf.columns)}')

[Info] Reading Data Files.....
2023-05-12 14:45:10.016393
[Info] df size: 2680028163, 8


In [7]:
sdf2 = sdf.alias('1')
# sdf = sdf2.alias('1')

In [9]:
# drop na domain rows
sdf = sdf.na.drop(subset=['SSL_CERT_DOMAIN'])
# print(f'[Info] df size after dropped na in Domain Column: {sdf.count()}, {len(sdf.columns)}')

# print(f'[Info] End of Reading Data Files')
# print(datetime.now())

### 3. -------------------------------------- Transform Data and Create Columns --------------------------------------

# create necessary columns for grouping
sdf = sdf.withColumn('ACCESS_DATE', F.to_date(F.col('TIME_START'), 'yyyyMMddHHmmss')) \
                     .withColumn('SSL_CERT_DOMAIN', F.upper(F.col('SSL_CERT_DOMAIN'))) \
                     .withColumn('START_TIMESTAMP', F.to_timestamp(F.col('TIME_START'),'yyyyMMddHHmmss')) \
                     .withColumn('END_TIMESTAMP', F.to_timestamp(F.col('TIME_END'),'yyyyMMddHHmmss')) \
                     .withColumn('SECONDS', F.col('END_TIMESTAMP').cast('long') - F.col('START_TIMESTAMP').cast('long')) \
                     .withColumn('MINUTES', F.col('SECONDS')/60) \
                     .withColumn('START_HOUR', F.hour(F.col('START_TIMESTAMP'))) \
                     .withColumn('START_INTERVAL_15MIN', F.floor(((F.substring(F.col('TIME_START'), 9,2).cast('int') * 60 + F.substring(F.col('TIME_START'), 11,2).cast('int'))/15))) \
                     .withColumn('START_INTERVAL_30MIN', F.floor(((F.substring(F.col('TIME_START'), 9,2).cast('int') * 60 + F.substring(F.col('TIME_START'), 11,2).cast('int'))/30)))

# add interval col
# interval_to_cut = [15, 30]
# for i in interval_to_cut:
#     sdf = sdf.withColumn(f'START_INTERVAL_{i}min', (F.substring(F.col('TIME_START'), 9,10).cast(int) * 60 + F.substring(F.col('TIME_START'), 11,12).cast(int))//i)

# group by SUBR_NUM, ACCESS_DATE, SSL_CERT_DOMAIN
sdf = sdf.groupBy('MSISDN', 'ACCESS_DATE', 'SSL_CERT_DOMAIN').agg(
                F.count("*").alias('TOTAL_HIT'), \
                F.collect_set(F.col('START_HOUR')).alias('START_HOUR_SET'), \
                F.sum('BYTE_COUNT_UP').alias('BYTE_COUNT_UP'), \
                F.sum('BYTE_COUNT_DOWN').alias('BYTE_COUNT_DOWN'), \
                F.sum('SECONDS').alias('SECONDS'), \
                F.sum('MINUTES').alias('MINUTES'), \
                F.collect_set(F.col('START_INTERVAL_15MIN')).alias('START_INTERVAL_15MIN_SET'), \
                F.collect_set(F.col('START_INTERVAL_30MIN')).alias('START_INTERVAL_30MIN_SET'), \
)

In [10]:
# sdf.limit(1).toPandas()

In [11]:
# sdf.count()

In [10]:
# add last few columns
sdf = sdf.withColumn('SUBR_NUM', when(F.col('MSISDN').startswith('852'), expr("substring(MSISDN, 4, length(MSISDN))")).otherwise(F.col('MSISDN'))) \
                     .withColumn('BYTE_COUNT', sum([F.col('BYTE_COUNT_UP'), F.col('BYTE_COUNT_DOWN')])) \
                     .withColumn('KB_COUNT', F.col('BYTE_COUNT')/byte_to_kb_ratio) \
                     .withColumn('KB_COUNT_UP', F.col('BYTE_COUNT_UP')/byte_to_kb_ratio) \
                     .withColumn('KB_COUNT_DOWN', F.col('BYTE_COUNT_DOWN')/byte_to_kb_ratio) \
                     .withColumn('HOUR_BINARY', udf_hourly_string(F.col('START_HOUR_SET'))) \
                     .withColumn('date_id', F.date_format(F.col('ACCESS_DATE'), "yyyyMMdd")) \
                     .withColumn('15MIN_BINARY', udf_15mins_string(F.col('START_INTERVAL_15MIN_SET'))) \
                     .withColumn('30MIN_BINARY', udf_30mins_string(F.col('START_INTERVAL_30MIN_SET')))

# reorder
#     sdf = sdf.select('SUBR_NUM', 'MSISDN', 'ACCESS_DATE', 'SSL_CERT_DOMAIN', 'BYTE_COUNT', 'KB_COUNT', 'HOUR_BINARY', 'TOTAL_HIT', 'BYTE_COUNT_UP', 'KB_COUNT_UP', 'BYTE_COUNT_DOWN', 'KB_COUNT_DOWN', 'SECONDS', 'MINUTES', 'date_id')
sdf = sdf.select(F.col('SUBR_NUM').cast("bigint").alias("subr_num"), \
                 F.col('SSL_CERT_DOMAIN').alias("ssl_cert_domain"), \
                 F.col('BYTE_COUNT').cast("bigint").alias("byte_count"), \
                 udf_b2int(F.col('HOUR_BINARY')).cast("int").alias("hour_binary"), \
                 F.col("TOTAL_HIT").cast("bigint").alias("total_hit"), \
                 'date_id', \
                 F.col('15MIN_BINARY').cast("string").alias("15min_binary"),
                 F.col('30MIN_BINARY').cast("string").alias("30min_binary"),
                 # udf_b2int(F.col('15MIN_BINARY')).cast("int").alias("15min_binary"), \
                 # udf_b2int(F.col('30MIN_BINARY')).cast("int").alias("30min_binary"),
                )

In [11]:
# filter only today's data
sdf = sdf.filter(F.col('date_id') == time_stamp)

# lower case column names
#     for col in sdf3.columns:
#         sdf3 = sdf3.withColumnRenamed(col, col.lower())

In [12]:
sdf.count()

259795153

In [14]:
# # filter only today's data
# sdf = sdf.filter(F.col('date_id') == time_stamp)

# # lower case column names
# #     for col in sdf.columns:
# #         sdf = sdf.withColumnRenamed(col, col.lower())

In [13]:
# ### 4. -------------------------------------- Save to parquet --------------------------------------

# # print(f'[Info] Start Transforming Data......')
# # print(datetime.now())

# # save to parquet
sdf.write.mode('append').partitionBy('date_id').parquet('/app/sna_bdmrdev03/ws-simon/data/weblog_15min/')

# # print(f'[Info] Saved Data to {pipeline_output_dir}')
# # print(datetime.now())

In [14]:
sdf_test = spark.read.parquet('/app/sna_bdmrdev03/ws-simon/data/weblog_15min/')

In [15]:
sdf_test.count()

259794791

In [16]:
sdf_test.limit(5).toPandas()

subr_num                      ssl_cert_domain  byte_count  hour_binary  \
0  19100044561        CONNECTIVITYCHECK.GSTATIC.COM        8330          448   
1     98538482                 CDN.DOUBLEVERIFY.COM      177018          128   
2  19100046612                      M.YAP.YAHOO.COM      472010      1572864   
3     98538482                       DIS.CRITEO.COM      132962       131200   
4  19100082639  GEOMOBILESERVICES-PA.GOOGLEAPIS.COM       11590       131072   

   total_hit  \
0         10   
1          3   
2          3   
3         16   
4         10   

                                                                                       15min_binary  \
0  000000000000000000000000000000000000000000000000000000000000000110000001000000000000000000000000   
1  000000000000000000000000000000000000000000000000000000000000000000100000000000000000000000000000   
2  000000000000001100010000000000000000000000000000000000000000000000000000000000000000000000000000   
3  000000000000000000000000001100000000000000000000000000000000000001100000000000000000000000000000   
4  000000000000000000000000100000000000000000000000000000000000000000000000000000000000000000000000   

                                       30min_binary   date_id  
0  000000000000000000000000000000011001000000000000  20230511  
1  000000000000000000000000000000000100000000000000  20230511  
2  000000010100000000000000000000000000000000000000  20230511  
3  000000000000010000000000000000001100000000000000  20230511  
4  000000000000100000000000000000000000000000000000  20230511

In [19]:
spark.stop()
sc.stop()